<h1><center>Calculating Portfolio VaR and understanding which Asset is the major contributor to the portfolio risk</center></h1>



In [2]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import math

In [3]:
#generating random numbers for Returns of the three assets in the portfolio
import numpy as np

def draw_returns(N):

    # coin flips
    normal_year = np.random.binomial(1, 0.9, N)

    # draw for normal years
    mu = np.array([0.05, 0.05, 0.05])
    Sigma = np.array([[0.09, 0.012, 0.021], [0.012, 0.16, 0.028], [0.021, 0.028, 0.49]])
    normal_ret = np.random.multivariate_normal(mu, Sigma, N)

    # draws for special years
    mu = np.array([-0.1, -0.1, -0.1])
    Sigma = np.array([[0.36, 0.24, 0.42], [0.24, 0.64, 0.56], [0.42, 0.56, 1.96]])
    special_ret = np.random.multivariate_normal(mu, Sigma, N)

    # combine
    ret = normal_ret
    for i in range(N):
        if normal_year[i] == 0:
            ret[i,:] = special_ret[i,:]

    return(ret)

In [5]:
np.random.seed(1234)
# Generate portfolio profits (in 1000 $)
rets = draw_returns(10000)
def portfolio_profit(returns, holdings):
    return(np.dot(returns, holdings))
holdings_Q1 = np.array([2000, 6000, 2000]) # in 1000 $
# Let W0 = 0
gains = portfolio_profit(rets, holdings_Q1)
def VaR_from_gains(gains, c=0.99):
    n = len(gains)
    sort = sorted(gains)
    VaR = -sort[math.ceil(n * (1-c))-1]
    return(VaR)
VaR_Q1 = VaR_from_gains(gains)
VaR_Q1
# VaR is around

10835.348056933195

## Calculating VaR of the Portfolio

In [4]:
np.random.seed(123)
returns = draw_returns(1000000)

#Fetching Returns of Asset A, B and C
returns_A = returns[:,0]
returns_B = returns[:,1]
returns_C = returns[:,2]

# Portfolio Return : Because VaR is the one tailed mesaure and Normal distribution is symmetric
returns_portfolio = returns_A * 2000000 + returns_B * 6000000 + returns_C * 2000000
mean_return = np.mean(returns_portfolio / 10000000)
print("The mean return from the simulation and considering intial investment is " + str(mean_return))

The mean return from the simulation and considering intial investment is 0.03526255052511689


In [5]:
# Portfolio Standard Deviation 
weights = np.array([0.2,0.6,0.2])
cov_matrix = np.cov([returns_A,returns_B,returns_C])
std_portfolio = np.sqrt(weights.T.dot(cov_matrix).dot(weights))
print("The standard deviation of the portfolio from the simulation and considering intial investment is " + str(std_portfolio))

The standard deviation of the portfolio from the simulation and considering intial investment is 0.37356279735760856


#### The VaR of the Portfolio will be calculated using the total Intital Investment ie 10 Million by multiplying the intial investment value to returns_portfolio and std_portfolio

In [6]:
# VaR of the Portfolio for (c = 99%) worst case
initial_investment = 10000000
mean = (1+mean_return) * initial_investment
std = std_portfolio * initial_investment

F_inverse_95 = st.norm.ppf(0.05, loc = mean, scale = std)

VaR_portfolio_95 = initial_investment - F_inverse_95
print("The VaR of the Portfolio of 95% worst case is " + str(-1 *VaR_portfolio_95))

F_inverse_99 = st.norm.ppf(0.01, loc = mean, scale = std)

VaR_portfolio_99 = (initial_investment - F_inverse_99)
print("The VaR of the Portfolio of 99% worst case is " + str(-1 *VaR_portfolio_99))

The VaR of the Portfolio of 95% worst case is -5791935.716026836
The VaR of the Portfolio of 99% worst case is -8337744.689285051


### Calculating DVaR for each asset

### For Asset A
DVaR for Asset A = Base VaR - VaR with increase of delta in Asset A / delta

In [7]:
delta = 1000

# Calculating DVaR for asset A :
returns_da = returns_A * 2001000 + returns_B * 6000000 + returns_C * 2000000
#returns_portfolio = returns_portfolio + 1
mean_return_da = np.mean(returns_da / 10001000)

# Standard Deviation Port_da
weights_da = np.array([2001000/10001000,6000000/10001000, 2000000/10001000])
cov_matrix = np.cov([returns_A,returns_B,returns_C])
std_portfolio_da = np.sqrt(weights_da.T.dot(cov_matrix).dot(weights_da))

# VaR of the Portfolio for (c = 99%) worst case
new_investment = 10001000
mean_da = (1+mean_return_da) * new_investment
std_da = std_portfolio_da * new_investment

F_inverse_99_da = st.norm.ppf(0.01, loc = mean_da, scale = std_da)

VaR_portfolio_99_da = new_investment - F_inverse_99_da
print("The VaR of the Portfolio with increase of delta in Asset A for 99% worst case is " + str(-1 *VaR_portfolio_99_da))

#Dvar for Asset A = (Base Var - Var with increase of delta in Asset A) / delta
DVaR_A = (VaR_portfolio_99 - VaR_portfolio_99_da) / delta
print("The DVaR for Asset A is " + str(DVaR_A))

The VaR of the Portfolio with increase of delta in Asset A for 99% worst case is -8338073.267743992
The DVaR for Asset A is -0.3285784589406103


### For Asset B
DVaR for Asset B = Base VaR - VaR with increase of delta in Asset B / delta

In [8]:
delta = 1000

# Calculating DVaR for asset B :
returns_db = returns_A * 2000000 + returns_B * 6001000 + returns_C * 2000000
#returns_portfolio = returns_portfolio + 1
mean_return_db = np.mean(returns_db / 10001000)

# Standard Deviation Port_da
weights_db = np.array([2000000/10001000,6001000/10001000, 2000000/10001000])
cov_matrix = np.cov([returns_A,returns_B,returns_C])
std_portfolio_db = np.sqrt(weights_db.T.dot(cov_matrix).dot(weights_db))

# VaR of the Portfolio for (c = 99%) worst case
new_investment = 10001000
mean_db = (1+mean_return_db) * new_investment
std_db = std_portfolio_db * new_investment

F_inverse_99_db = st.norm.ppf(0.01, loc = mean_db, scale = std_db)

VaR_portfolio_99_db = new_investment - F_inverse_99_db
print("The VaR of the Portfolio with increase of delta in Asset B for 99% worst case is " + str(-1 *VaR_portfolio_99_db))

#Dvar for Asset B = (Base Var - Var with increase of delta in Asset B) / delta
DVaR_B = (VaR_portfolio_99 - VaR_portfolio_99_db) / delta
print("The DVaR for Asset B is " + str(DVaR_B))

The VaR of the Portfolio with increase of delta in Asset B for 99% worst case is -8338642.461460425
The DVaR for Asset B is -0.8977721753735095


### For Asset C
DVaR for Asset C = Base VaR - VaR with increase of delta in Asset C / delta

In [9]:
delta = 1000

# Calculating DVaR for asset C :
returns_dc = returns_A * 2000000 + returns_B * 6000000 + returns_C * 2001000
#returns_portfolio = returns_portfolio + 1
mean_return_dc = np.mean(returns_dc / 10001000)

# Standard Deviation Port_da
weights_dc = np.array([2000000/10001000,6000000/10001000, 2001000/10001000])
cov_matrix = np.cov([returns_A,returns_B,returns_C])
std_portfolio_dc = np.sqrt(weights_dc.T.dot(cov_matrix).dot(weights_dc))

# VaR of the Portfolio for (c = 99%) worst case
new_investment = 10001000
mean_dc = (1+mean_return_dc) * new_investment
std_dc = std_portfolio_dc * new_investment

F_inverse_99_dc = st.norm.ppf(0.01, loc = mean_dc, scale = std_dc)

VaR_portfolio_99_dc = new_investment - F_inverse_99_dc
print("The VaR of the Portfolio with increase of delta in Asset C for 99% worst case is " + str(-1 *VaR_portfolio_99_dc))

#Dvar for Asset C = (Base Var - Var with increase of delta in Asset C) / delta
DVaR_C = (VaR_portfolio_99 - VaR_portfolio_99_dc) / delta
print("The DVaR for Asset C is " + str(DVaR_C))

The VaR of the Portfolio with increase of delta in Asset C for 99% worst case is -8338891.860042034
The DVaR for Asset C is -1.1471707569826395


### DVaR results for Asset A, B and C

In [10]:
print("The DVaR for Asset A is " + str(DVaR_A))
print("The DVaR for Asset B is " + str(DVaR_B))
print("The DVaR for Asset C is " + str(DVaR_C))

The DVaR for Asset A is -0.3285784589406103
The DVaR for Asset B is -0.8977721753735095
The DVaR for Asset C is -1.1471707569826395


### CVaR for each Asset A,B and C

$C VaR_{i}$ = $x_{i}$ * $\delta $VaR_${i}$ / $\delta $x_${i}$

In [11]:
CVaR_A = 2001000*(DVaR_A)
CVaR_B = 6001000*(DVaR_B)
CVaR_C = 2001000*(DVaR_C)

print("The CVaR for Asset A is " + str(CVaR_A))
print("The CVaR for Asset B is " + str(CVaR_B))
print("The CVaR for Asset C is " + str(CVaR_C))

The CVaR for Asset A is -657485.4963401612
The CVaR for Asset B is -5387530.824416431
The CVaR for Asset C is -2295488.6847222615


In [12]:
print("The sum of CVaRs for the three assets is " + str(CVaR_A + CVaR_B + CVaR_C))
print("The VaR of the orginial portfolio with 2M in Asset A, 6M in Asset B and 2M in asset C is " + str(VaR_portfolio_99 * -1))

The sum of CVaRs for the three assets is -8340505.005478853
The VaR of the orginial portfolio with 2M in Asset A, 6M in Asset B and 2M in asset C is -8337744.689285051


#### As we see from the above output that sum of CVaRs of Assets A,B and C is very close to the VaR of the orignial Portfolio.

#### Also, Component VaR is a good measure of how much each asset contributes to the overall risk of the portfolio.From the CVaR output above we see that asset responsible for the most risk of the portfolio is Asset B - and it is also because the amount invested in asset B is 6 Million which is double of the amount invested in Asset A and Asset C. 

#### Part (2)

When you approximate the derivatives involved in DVaR and CVaR, vary the size of the position change you use. What do you observe when you change the value? Report a graph that shows the effect of different size of the step for the derivative. Explain what is happening.



In [ ]:
# Coming up with a range for weights

allocations = []
for al1 in range(0,100, 5):
    for al2 in range(0,100, 5): 
        for al3 in range(0, 100, 5):
            if al1+al2+al3 == 100:
                allocations.append([al1, al2, al3])
allocations = (pd.DataFrame(alloc)/100).to_numpy()

In [5]:
# Coming up with VaRs for a range of weights
dd= np.arange(0.1,1000,0.1)
w_A = []
w_B = []
w_C = []
for d in dd:
    w_A.append((2000000+d)/(2000000+d+6000000+2000000))
    w_B.append((6000000)/(2000000+d+6000000+2000000))
    w_C.append(2000000/(2000000+d+6000000+2000000))
    
port_d1 = []
for a in range(0,9999,1):
    port_d1.append((returns_A[0:9999]*np.array(w_A[a]))+(returns_B[0:9999]*np.array(w_B[a]))+(returns_C[0:9999]*np.array(w_C[a])))

In [ ]:
port_ret = (returns_A[0:9999]*0.2)+(returns_B[0:9999]*0.6)+(returns_C[0:9999]*0.2)
port_dvar = []
for j in range(0,9999):
    port_dvar.append(port_ret - port_d1[j])

DVaR = []
for d in dd[0:9999]:
    for k in range(0,9999):
        DVaR.append(np.array(port_dvar[k])/d)

#### You change your portfolio to 3m in asset A, 5m in asset B and 2m in asset C. What are the CVaR and DVaR for asset C?
Part (3)

In [13]:
delta = 1000

# Calculating DVaR for asset C :
returns_dc1 = returns_A * 3000000 + returns_B * 5000000 + returns_C * 2001000
#returns_portfolio = returns_portfolio + 1
mean_return_dc1 = np.mean(returns_dc1 / 10001000)

# Standard Deviation Port_da
weights_dc1 = np.array([3000000/10001000,5000000/10001000, 2001000/10001000])
cov_matrix = np.cov([returns_A,returns_B,returns_C])
std_portfolio_dc1 = np.sqrt(weights_dc1.T.dot(cov_matrix).dot(weights_dc1))

# VaR of the Portfolio for (c = 99%) worst case
new_investment = 10001000
mean_dc1 = (1+mean_return_dc1) * new_investment
std_dc1 = std_portfolio_dc1 * new_investment

F_inverse_99_dc1 = st.norm.ppf(0.01, loc = mean_dc1, scale = std_dc1)

VaR_portfolio_99_dc1 = new_investment - F_inverse_99_dc1
print("The VaR of the Portfolio with increase of delta in Asset C for 99% worst case is " + str(VaR_portfolio_99_dc1))

#Dvar for Asset C = (Base Var - Var with increase of delta in Asset C) / delta
DVaR_C1 = (VaR_portfolio_99 - VaR_portfolio_99_dc1) / delta
print("The DVaR for Asset C is " + str(DVaR_C1))

The VaR of the Portfolio with increase of delta in Asset C for 99% worst case is 7834732.552339262
The DVaR for Asset C is 503.01213694578877


In [14]:
#Dvar for Asset C = (Base Var - Var with increase of delta in Asset C) / delta
DVaR_C1 = (VaR_portfolio_99 - VaR_portfolio_99_dc1) / delta
print("The DVaR for Asset C is " + str(DVaR_C1))


CVaR_C1 = 2001000*(DVaR_C1)
print("The CVaR for Asset C is " + str(CVaR_C1))

The DVaR for Asset C is 503.01213694578877
The CVaR for Asset C is 1006527286.0285233


#### Conclusion - In this part (3) of the question we changed the asset allocation of Asset A and Asset B while the asset allocation of Asset C of 2 million remains unchanged. Despite this we see from the above output that DVaR and CVaR of asset C has changed. This leads to a very important conclusion that the DVaR and CVaR of the asset are not a characteristic of a particular asset but characterisitc of an asset within a portfolio and both will change from portfolio to portfolio and will also depend on the asset allocations of other assets.